# Lab E: コンピュータ・ビジョン入門

このノートブックでは、Racecarのカメラを利用して色を判別することで物体を識別し、その中心と面積を抽出する方法を学びます。

このノートブック全体を通して **<font style="color:red">太い赤字で書かれた文章</font>** は、実行する前にその下のコードブロックを編集して正しいコードを書く必要があります。

## 目次
1. [はじめに](#GettingStarted)
1. [写真の撮影](#TakingPhotos)
1. [カラーフォーマット](#ColorFormats)
1. [マスク](#Masks)
1. [輪郭の検出](#FindingContours)
1. [輪郭の中心](#ContourCenter)
1. [輪郭の面積](#ContourArea)
1. [数値処理](#ProcessingNumericValues)
1. [比例制御](#ProportionalControl)

<a id="GettingStarted"></a>
## 1. はじめに

**<font style="color:red">もしシミュレータを利用して開発を進める場合は、 `isSimulation` を `True` に設定します </font>**。 実際のマシンを利用する場合は、 `isSimulation` を `False` のままにしてください。

In [ ]:
# TODO: 必要に応じてisSimulationを更新する
isSimulation = True

次に、Pythonライブラリ(`cv`, `numpy`, など)や、Racecarライブラリ(`racecar_core`)など、このノートブックの実行に必要なライブラリをインポートします。

In [ ]:
# Pythonライブラリのインポート
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from nptyping import NDArray
from typing import Any, Tuple, List, Optional

# Racecarライブラリのインポート
import sys
sys.path.append("../../library")
import racecar_core

以下の関数や機能はこのノートブックでの開発を補助するものです。
実行しておいてください。

In [ ]:
def show_image(image: NDArray) -> None:
    """
    Jupyter Notebookにカラー画像を表示します。
    """
    plt.imshow(cv.cvtColor(image, cv.COLOR_BGR2RGB))
    plt.show()

    
def draw_contour(
    image: NDArray,
    contour: NDArray,
    color: Tuple[int, int, int] = (0, 255, 0)
) -> None:
    """
    指定された画像に輪郭を描画する。

    Args:
        image: 輪郭を描画する画像
        contour: 画像に描画する輪郭
        color: BGR形式でして逸された輪郭を描画する色
    """
    cv.drawContours(image, [contour], 0, color, 3)

    
def draw_circle(
    color_image: NDArray[(Any, Any, 3), np.uint8],
    center: Tuple[int, int],
    color: Tuple[int, int, int] = (0, 255, 255),
    radius: int = 6,
) -> None:
    """
    指定された画像に円を描く

    Args:
        color_image: 輪郭を描画する色画像
        center: 画像の中心ピクセル (行, 列)
        color: BGR形式で指定された円を描く色
        radius: ピクセル単位の円の半径
    """
    # cv.circle は(列, 行)の形式で円の中心を指定します
    cv.circle(color_image, (center[1], center[0]), radius, color, -1)

    
def show_color_bgr(blue: int, green: int, red: int) -> None:
    """
    BGR形式で指定された色を表示する
    """
    rectangle = plt.Rectangle((0,0), 50, 50, fc=(red/255, green/255, blue/255))
    plt.gca().add_patch(rectangle)
    plt.show()

    
def show_color_hsv(hue: int, saturation: int, value: int) -> None:
    """
    HSV形式で指定された色を表示する
    """
    # HSVからBGR形式に変換する
    hsv = np.array([[[hue, saturation, value]]], np.uint8)
    bgr = cv.cvtColor(hsv, cv.COLOR_HSV2BGR)
    
    show_color_bgr(bgr[0][0][0], bgr[0][0][1], bgr[0][0][2])

最後に、Racecarオブジェクトを作成します。このステップで失敗した場合は`isSimulation` が正しい値であることを確認してください。

In [ ]:
# Racecar オブジェクトの作成
rc = racecar_core.create_racecar(isSimulation)

<a id="TakingPhotos"></a>
## 2. 写真の撮影

Jupyter Notebookでは、`rc.camera.get_color_image_async()` を利用して車のカメラで写真を撮ることができます。Jupyter Notebookではなく、実機側で写真を撮影する場合は、`rc.camera.get_color_image()` を使う必要があります。

Racecarが今現在何を見ているのかを確認してみましょう。

In [ ]:
# 写真を撮影し画面に表示する
image = rc.camera.get_color_image_async()
show_image(image)

カラー画像は3次元のnumpy配列として格納される: 

* **0次元目**: ピクセル行、上から下へのインデックス
* **1次元目**: ピクセル行、左から右へのインデックス
* **2次元目**: ピクセルの色値。青、緑、赤の順で、それぞれ0(色が全くない)から255(その色の最大値)まである

画像の真ん中のピクセルの色を見てみましょう。

In [ ]:
# 中心の行と列を計算する
row = rc.camera.get_height() // 2
col = rc.camera.get_width() // 2

# 青、緑、赤の各色の値を抽出して表示する
blue = image[row][col][0]
green = image[row][col][1]
red = image[row][col][2]

print("blue:", blue)
print("green:", green)
print("red:", red)

# 同じ色を画面に表示する
show_color_bgr(blue, green, red)

**<span style="color:red">次のコードブロックの `row` と `col` を更新して、上から2/3、右から1/4のピクセルを表示する</span>**

In [ ]:
# TODO Part 1: 希望の行と列を計算する 
row = 
col = 

# 青、緑、赤の各色の値を抽出して表示する
blue = image[row][col][0]
green = image[row][col][1]
red = image[row][col][2]

print("blue:", blue)
print("green:", green)
print("red:", red)

# 同じ色を画面に表示する
show_color_bgr(blue, green, red)

<a id="ColorFormats"></a>
## 3. カラーフォーマット
カメラで撮影された画像はデフォルトでは、青・緑・赤（BGR）形式で保存されます。しかし、色情報に基づいて物体を認識する場合は、各チャンネルがいかに対応する、色相・彩度・値（HSV）フォーマットを使用するほうがはるかに簡単です:

* **色相(Hue)** (0 ~ 180): カラーホイール上で赤-橙-黄-緑-青-紫-赤の順に並んだ色
* **彩度(Saturation)** (0 ~ 255): 色に加えられる白の量。0は純粋な白で、255は白を加えない純粋な色
* **値(Value)** (0 ~ 255): 色に加えられる黒の量。0は純粋な黒で、255は黒を加えない純粋な色

彩度と値は照明のあたり方によって変化するが、色相は照明に関係なくほとんど変わりません。検出しようとしている物体の色相に注目する事で、異なる照明環境でもその物体を見つける事ができます。

次のウィジェットを利用することで、BGRとHSVフォーマットの様々な色を試す事ができます。  **<font style="color:red">両方のフォーマットで、次の色を生成する値を見つけてください: オレンジ、ピンク、濃い緑、黄色、灰色</font>**

In [ ]:
# BGR color
widgets.interact(show_color_bgr,
                 blue=widgets.IntSlider(0, 0, 255, continuous_update=False),
                 green=widgets.IntSlider(0, 0, 255, continuous_update=False),
                 red=widgets.IntSlider(0, 0, 255, continuous_update=False));

In [ ]:
# HSV color
widgets.interact(show_color_hsv,
                 hue=widgets.IntSlider(0, 0, 180, continuous_update=False),
                 saturation=widgets.IntSlider(255, 0, 255, continuous_update=False),
                 value=widgets.IntSlider(255, 0, 255, continuous_update=False));

<a id="Masks"></a>
## 4. マスク
車の視野内の物体をその色に基づいて識別することに挑戦してみましょう。具体的には、HSVの上限と下限を定義することで、画像における特定の色の範囲に含まれる部分のみを切り出します。これを使って画像から抜き出す部分は白、抜き出さない部分は黒という特殊な画像 *マスク* を作成します。

**<font style="color:red">画像を受け取り、hsv_lowerとhsv_upperの間の領域のマスクを返す `get_mask` 関数を完成させましょう。</font>**  次のようなOpenCVの関数を利用してください:

* [`cvtColor`](https://docs.opencv.org/4.2.0/d8/d01/group__imgproc__color__conversions.html#ga397ae87e1288a81d2363b61574eb8cab): BGRからHSVのように、画像をある色形式から別の色形式に変換します
* [`inRange`](https://docs.opencv.org/4.2.0/d2/de8/group__core__array.html#ga48af0ab51e36436c5d04340e036ce981): 下限となる色と上限となる色に基づいて画像からマスクを作成します

In [ ]:
def get_mask(
    image: NDArray[(Any, Any, 3), np.uint8],
    hsv_lower: Tuple[int, int, int],
    hsv_upper: Tuple[int, int, int]
) -> NDArray[Any, Any]:
    """   
    hsv_lowerとhsv_upperの間にある画像領域すべてを含むマスクを返す
    
    Args:
        image: マスクのもととなるBGR形式の画像
        hsv_lower: マスクに含めるHSV値の下限
        hsv_upper: マスクに含めるHSV値の上限
    """
    # hsv_lowerとhsv_uppperをnumpy配列に変換し、OpenCVで使えるようにする
    hsv_lower = np.array(hsv_lower)
    hsv_upper = np.array(hsv_upper)
    
    # TODO Part 2: cv.cvtColor関数を使用して、BGRカラーをHSVカラーに変換する
    
    # TODO Part 3: cv.inRange関数を使用して、正しい範囲の領域を協調表示する
    
    return _____

`get_mask` 関数を使って画像内のコーンを分離してみましょう。車の視界の中にコーンを置いて、次のブロックで写真を撮ります。

In [ ]:
image = rc.camera.get_color_image_async()
show_image(image)

次に、`get_mask` 関数を使用して、コーンだけを含むマスク画像を作成します。現時点では、`hsv_lower` と `hsv_upper` はHSVの全体の値を含むので、マスクは画像全体を含むことになります。**<font style="color:red">マスク画像がコーンだけを含むようになるまで、 `hsv_lower` と `hsv_upper` の値を調整してください。</font>**

**ヒント:**

* HSVカラーを視覚化るするのには、[カラーフォーマット](#ColorFormats) 節で利用したHSVカラーウィジェットを使います。
* 画像を画像編集ソフト(gimp, ペイントなど)にコピーし、スポイト(カラーピッカー)ツールを使って、コーン内のピクセルのHSV値を調べてみましょう。
* 彩度(Saturation)と値(Value)は照明によって大きく変化するが、色相はある物体に対してほぼ一定に保たれます。彩度や値には広い範囲を、色相(Hue)には狭い範囲を設定してみてください。

In [ ]:
# TODO Part 4: マスク画像がコーンだけを含むようになるまで、hsv_lowerとhsv_uppperの値を調整する
hsv_lower = (0, 50, 50)
hsv_upper = (20, 255, 255)

mask = get_mask(image, hsv_lower, hsv_upper)
show_image(mask)

このマスクは、`hsv_lower` と `hsv_upper` の間にある部分のみを残すためのフィルターとして、元画に適用する事ができます。

In [ ]:
masked_image = cv.bitwise_and(image, image, mask=mask)
show_image(masked_image)

<a id="FindingContours"></a>
## 5. 輪郭の検出

マスクができたので、マスク内の各物体の周囲に _輪郭(Contours)_ と呼ばれる輪郭線(アウトライン)を作成することができます。この輪郭線を用いて、画像内の最も大きな物体を特定し、その物体のサイズと位置を計算します。

まず、OpenCVの関数 [`findContours`](https://docs.opencv.org/4.2.0/d3/dc0/group__imgproc__shape.html#gadf1ad6a0b82947fa1fe3c3d497f260e0) を使って、マスク内の各物体を囲む輪郭のリストを作成します。

In [ ]:
def find_contours(mask: NDArray) -> List[NDArray]:
    """
    マスク内のすべてのオブジェクトを囲む輪郭のリストを返します
    """
    return cv.findContours(mask, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)[0]

`find_contours` は、 `hsv_lower` と `hsv_upper` の間に入る物体が複数存在する場合に、複数の輪郭を含むリストを返します。これは、画像内に複数のコーンがある場合や、コーンと同じような色を持つ物体がほかにもある場合に発生します。

そこで、最大の輪郭を識別するための補助関数を作成してみましょう。また、この補助関数は最小サイズ(`30ピクセル` など)以下の輪郭を無視するような機能を持つ必要があります。

**<font style="color:red">`get_largest_contour` 関数を`min_area`より大きい最大の輪郭を返すように加筆修正してください。また、もしそのような輪郭がない場合は`None`を返すようにしてください。</font>**  輪郭に含まれるピクセル数を求めるには、OpenCVの [`contourArea`](https://docs.opencv.org/4.2.0/d3/dc0/group__imgproc__shape.html#ga2c759ed9f497d4a618048a2f56dc97f1) 関数を使用するのが良いでしょう。

In [ ]:
def get_largest_contour(contours: List[NDArray], min_area: int = 30) -> Optional[NDArray]:
    """
    min_area よりも大きなサイズを持つ最大の輪郭を見つける

    Args:
        contours: 画像から検出された輪郭のリスト
        min_area: 考慮する最小輪郭(ピクセル数)

    Returns:
        min_area より大きい輪郭がない場合はNoneを返す
    """
    if len(contours) == 0:
        # TODO Part 5: 輪郭のリストが空の場合、何を返すべきでしょうか？
        return _____
    
    # TODO Part 6: min_area より大きい輪郭がない場合はNoneを返す

    return _____

それでは、作成した関数を試してみましょう。次のコードブロックは`find_contours` と `get_largest_contour`を使って最大輪郭を見つけて、画像上に描画します。これで画像の一番近いコーンを囲む緑色の輪郭が表示されるはずです。

In [ ]:
# 最大の輪郭を見つける
contours = find_contours(mask)
largest_contour = get_largest_contour(contours)

# 最大の輪郭を表示する
image_copy = np.copy(image)
draw_contour(image_copy, largest_contour)
show_image(image_copy)

<a id="ContourCenter"></a>
## 6. 輪郭の中心

輪郭の利点の1つは、輪郭を利用して物体の中心を簡単に計算できることです。具体的には輪郭の[_モーメント(Moments)_](https://en.wikipedia.org/wiki/Image_moment)を利用します。これは輪郭内のピクセルの加重平均に相当します。モーメント$M_{ij}$は下記の式で計算することができます。

```
def moment(i, j):
    sum = 0
    for pixel in contour:
        sum += pixel.x_position ** i + pixel.y_position ** j
    return sum
```

輪郭の中心を計算するには以下の種類のモーメントを利用します:

* $M_{00}$: 輪郭のピクセル数
* $M_{10}$: 輪郭の各ピクセルが右にどれだけ離れているかの和
* $M_{01}$: 輪郭の各ピクセルがどれだけ下にあるかの和

[重心(center of mass equation)](https://en.wikipedia.org/wiki/Center_of_mass)を用いると, $\frac{M['m10']}{M['m00']}$ で輪郭の水平方向の平均位置(列)が得られ、$\frac{M['m01']}{M['m00']}$ で垂直方向の平均位置(行)が得られる。

In [ ]:
def get_contour_center(contour: NDArray) -> Optional[Tuple[int, int]]:
    """
    画像から輪郭の中心を見つける

    Args:
        contour: 中心を求める輪郭

    Returns:
        輪郭の中心にあるピクセルの(行、列)、または輪郭が空の場合はNone
    """
    # 輪郭のモーメントを計算するようにOpenCVを利用する
    M = cv.moments(contour)

    # 輪郭が空かどうかを確認する
    if M["m00"] <= 0:
        return None

    # 輪郭の重心を計算する
    center_row = round(M["m01"] / M["m00"])
    center_column = round(M["m10"] / M["m00"])
    
    return (center_row, center_column)

これがうまくいったかどうかを確認するために、この計算された重心に点を描きます。コーンの中心に黄色い点が見えるはずです。

In [ ]:
center = get_contour_center(largest_contour)

# 輪郭の中心に円を描く
draw_circle(image_copy, center)
show_image(image_copy)

<a id="ContourArea"></a>
## 7. 輪郭の面積

`get_largest_contour` 関数を作成するとき、おそらくOpenCVの[`contourArea`](https://docs.opencv.org/4.2.0/d3/dc0/group__imgproc__shape.html#ga2c759ed9f497d4a618048a2f56dc97f1) 関数を利用したかと思います。輪郭の面積は物体がカメラからどれだけ離れているかを計算するのにも役立ちます。

このセクションでは、車からの距離を変えてコーンの面積を測定します。**<font style="color:red">ここまで作成した関数や例を使用して、以下のコードブロックを、写真を撮り、最大の輪郭を見つけ、輪郭の面積を表示し、輪郭のある画像を表示するように加筆修正してください。</font>**

In [ ]:
# TODO Part 7: 次のコードブロックを更新して、写真を撮り、最大の輪郭を見つけ、輪郭の面積を表示し、輪郭のある画像を表示します。

# TODO: 写真を撮る

# TODO: 最大の輪郭を見つける

# TODO: 最大の輪郭の面積を計算して、表示する

# TODO: 輪郭が描かれた画像を上に表示する

**<font style="color:red">コーンが車から次の距離にある時のコーンの輪郭面積を測定します: 40cm, 80cm, 120cm, 160cm, 200cm。測定した結果で次のコードブロックの`data`の情報を更新してください。</font>**

In [ ]:
# データのフォーマット (車からコーンまでの距離 [cm], 輪郭のピクセル数)
# TODO Part 8: 輪郭の面積をピクセル数で更新する
data = [
    (40, _____),
    (80, _____),
    (120, _____),
    (160, _____),
    (200, _____),
]

距離とコーンの面積の関係を見るために、このデータをプロットしてみよう。**この関係はどのように表現できるでしょうか？**

In [ ]:
# データを散布図でプロットする
data_t = np.transpose(data)
plt.scatter(data_t[0], data_t[1])
plt.title("Relationship between Distance and Contour Area")
plt.xlabel("Cone distance (cm)")
plt.ylabel("Contour area (pixels)")
plt.show()

<a id="ProcessingNumericValues"></a>
## 8. 数値処理

抽象的な見方をすると、多くのロボット工学の仕事は以下のようにようやくできます:

1. 1つもしくは複数のセンサー(画像、IMUデータ、LiDARスキャンなど)から生のデータを入力として受け取る
2. これらの生データの入力を意味のある数値(輪郭中心、輪郭面積、中心距離、現在速度んど)に変換する
3. これらの数値を使用して出力値(速度、角度など)を計算する
4. これらの出力値をコントローラ(モーターなど)に送る

前節では、ステップ2の例として、カラー画像(生データの入力)を中心点と輪郭面積(数値)にする例を2つ紹介しました。このセクションでは、ステップ3、つまりこれらの数値を出力値に変換するための2つの便利なツールを構築します。

### Clamp
出力値はある範囲に収まっているいなければならない事がよくあります。例えば、 `rc.drive.set_speed_angle()`に送られる速度と角度は$[-1, 1]$の範囲でなければなりません。*クランピング(Clamping)*とは、値を特定の範囲に強制的に押し込める事です。値が最小値より小さければ最小値を受け取り、値が最大値より大きければ最大値を受け取ります。

例えば、clamp関数は以下のような出力を返すはずです:

* `clamp(3, 0, 10) = 3`: $3$ は $0$ と $10$ の間の値なので、変更されません
* `clamp(-2, 0, 10) = 0`: $-2$ は $0$ より小さいので、最小値で飽和されます
* `clamp(11, 0, 10) = 10`: $11$ は $10$ より大きいので、最大値で飽和されます

**<font style="color:red">この動作を以下の`clamp`関数で実装する</font>**

In [ ]:
def clamp(value: float, min: float, max: float) -> float:
    """
    最小値と最大値の間の値にクランプする

    Args:
        value: クランプする入力
        min: 供される最小値
        max: 許容される最大値

    Returns:
        minとmaxの間で飽和した値
    """
    # TODO Part 9: 値が最小値と最大値の間であることを確認する
    return _____

それでは`clamp`関数をテストしてみましょう。

In [ ]:
# clamp関数のテスト
assert clamp(3, 0, 10) == 3
assert clamp(-2, 0, 10) == 0
assert clamp(11, 0, 10) == 10

次のウィジェットを使って、様々な入力を試す事ができます。

In [ ]:
widgets.interact(clamp,
                 value=widgets.FloatSlider(0, min=-2, max=2, step=0.1),
                 min=widgets.fixed(-1),
                 max=widgets.fixed(1));

### Remap Range

入力された値がある範囲に収まるかどうかで動作を決めたいことはよくあります。例えば、輪郭の中心のx座標に基づいて車の角度を設定したり、震度画像の中心距離に基づいて車の速度を設定したりします。1つのアプローチは、入力可能な値の範囲から出力可能な値の範囲に入力値を*再マッピング*することです。

例えば、$[0, 1]$の範囲を$[-1, 1]$に再マッピングするとします。$0$（古い最小値）は$-1$（新しい最小値）に、$0.5$（古い中点）は$0$（新しい中点）に、$1$（古い最大値）は$1$（新しい最大値）になります。

範囲は必ずしも「順序通り」である必要はありません。 例えば、小さい数を大きくしたい、あるいはその逆をしたい場合もあります。 これは、例えば$[0, 10]$を$[10, 0]$にリマップするなど、新しい範囲を反転させることで実現できます。 このようなリマップは、$2$は$8$に、$0$は$10$に、$6$は$4$に、 $20$は$-10$に、$-2$は$12$になります。

**<font style="color:red">`remap_range`で、`val`を古い範囲から新しい範囲にリマップするコードを作成してください。</font>**

In [ ]:
def remap_range(
    val: float,
    old_min: float,
    old_max: float,
    new_min: float,
    new_max: float,
) -> float:
    """
    ある範囲から別の範囲に値をリマップする

    Args:
        val: リマップされる古い範囲の数値
        old_min: 古い範囲の'下限'
        old_max: 古い範囲の'上限'
        new_min: 新しい範囲の'下限'
        new_max: 新しい範囲の'上限'

    Note:
        方向を反転させるとマッピングの符号が反転します。
        valはold_minとold_maxの間にあるとは限りません。
    """
    # TODO Part 10: valを新しい範囲にリマップします
    
    return _____

それでは `remap range` 関数をテストしてみましょう。

In [ ]:
# remap_range関数のテスト
assert remap_range(5, 0, 10, 0, 50) == 25
assert remap_range(5, 0, 20, 1000, 900) == 975
assert remap_range(2, 0, 1, -10, 10) == 30
assert remap_range(200, 0, 640, -1, 1) == -0.375

次のウィジェットを使って、様々な入力を試すことができます。  **出力$2.0$をもたらす、3つの異なる入力の組み合わせを見つけてみましょう。**

In [ ]:
widgets.interact(remap_range,
                 val=widgets.FloatSlider(0, min=-10, max=10, step=0.1),
                 old_min=widgets.FloatSlider(0, min=-10, max=10, step=0.1),
                 old_max=widgets.FloatSlider(5, min=-10, max=10, step=0.1),
                 new_min=widgets.FloatSlider(-1, min=-10, max=10, step=0.1),
                 new_max=widgets.FloatSlider(1, min=-10, max=10, step=0.1));

<a id="ProportionalControl"></a>
## 9. 比例制御

ロボット工学でよくある目標は、望ましい状態を維持することです。例えば、車からコーンが常に30cm離れていたいとか、速度を1.0m/sに保ちたいとかです。現在の状態が望ましい状態と異なる場合、私たちは車のコントローラ(アクセル・ブレーキなどのスロットルや、ステアリングなど)を使って望ましい状態に向かうように制御する必要があります。

定常状態を維持するもっとも簡単な方法の一つが、[proportional control](https://en.wikipedia.org/wiki/Proportional_control) です。この戦略は、現在の値と望ましい値との差に*比例*した変化を適用します。例えば、室温を20.0度に保ちたいとします。実際の温度がこの目標値と1度異なるごとに100単位の冷暖房を適用します。つまり:
* 現在の温度が$20.0$度なら、希望する温度にいるので、$0$単位の暖房をかける
* 現在の温度が$19.7$度なら、$30$単位の暖房をかける
* 現在の温度が$21.7度なら、$-170$単位の暖房をかけます

以下の入力を与えて `remap_range` を使うことで、比例制御の計算を行うことができます。
* `val`: 入力変数 (例: 現在の室温)
* `old_min`: 入力変数の潜在的な値 (例: 希望室温)
* `old_max`: 入力変数のもう一つの潜在的な値 (例: 希望室温より1度低い室温)
* `new_min`: 入力が`old_min`になった時の理想的な出力 (例: 暖房/冷房なし)
* `new_max`: 入力が`old_max`になった時の理想的な出力 (例: 100単位の暖房)

`min`と`max`は境界線ではないので、順番に並べる必要はありません。これらは単に線形関係を特徴づけるための2つの基準点です。

**<font style="color:red">`current_temp`の値を変えて試してみましょう。</font>**

In [ ]:
DESIRED_TEMP = 20
current_temp = 29.7 # TODO: 異なる値に変更して試してみましょう

# remap_rangeを使って、適用する暖房の単位あたりの出力を計算する
heating = remap_range(current_temp, DESIRED_TEMP, DESIRED_TEMP - 1, 0, 100)
print(f"{heating:.1f} units of heating")

出力がある範囲内に収まらなければならない場合、`clamp`を使ってこの範囲を守るように強制することができます。例えば、サーモスタットが暖房または冷房を最大500単位まで適用できるとします。次のコードブロックは、`clamp`を使ってこの制限を強制します。

**<font style="color:red">`current_temp`が15度以下または25度以上の場合に何が起こるか見てみましょう。</font>**

In [ ]:
clamped_heating = clamp(heating, -500, 500)
print(f"{clamped_heating:.1f} units of heating (after clamping)")

最後の課題では、比例仔魚を使用して、カラー画像内のコーンの位置に基づいて前輪の角度を決定します。具体的には次のような手順で車をコーンに向けて操舵します:

1. カラー画像を撮る
2. 色に基づいて科増をマスクし、コーンと想定される最大の輪郭を特定する
3. この輪郭の中心を見つけます
4. 輪郭と中心が描かれたカラー画像を表示します
5. 比例制御を使って、中心の列の位置を角度に変換します。つまり、中心から見て、コーンが左にどれだけ離れているかに比例して車は左に曲がり、コーンが右にどれだけどれだけ離れているかに比例して車は右に曲がるはずです。

**<font style="color:red">角度を計算するには、次のコードブロックの中で次のステップを実行します。</font>**

In [ ]:
# TODO Part 11: 比例制御を使用して、カラー画像内のコーンの位置に基づいて前輪の角度を決定させる

# カラー画像を撮影する
image = rc.camera.get_color_image_async()

# TODO: カラーマスクと最大輪郭からコーンを見つける
mask = get_mask(___, ___, ___)
contours = find_contours(___)
largest_contour = get_largest_contour(_____)

# TODO: 輪郭の中心を見つけ、その座標を表示する
center = _____
print("Contour center: ", center)

# 輪郭と中心が描かれた画像を表示する
image_copy = np.copy(image)
draw_contour(image_copy, largest_contour)
draw_circle(image_copy, center)
show_image(image_copy)

# TODO: 輪郭の中心を使って、[-1, 1]の範囲の角度を計算し、表示する
angle = _____
print("Angle: ", angle)

これで、`lab_e.py`でカラーカメラを利用してラインフォローを行う準備ができました。皆さんの幸運を祈ります！もし何かあれば遠慮なく質問してください！